## 填充和步幅

填充和步幅是两个控制卷积层输出大小的超参数。
- 填充：卷积时，图像的边缘部分只参与了一点点输出元素的生成，会丢失许多边界信息，同时使得2d大小减小过快，网络没法加深；
- 步幅：有时，我们可能希望大幅降低图像的宽度和高度。例如，如果我们发现原始的输入分辨率十分冗余。

In [2]:
import torch
from torch import nn

### 1.填充

- 填充：在输入周围添加额外的0行/0列；
- 在许多情况下，我们需要使输入和输出具有相同的高度和宽度；
- 卷积神经网络中卷积核的高度和宽度通常为奇数，例如1、3、5或7。好处：
  - 保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列。
  - 提供了书写上的便利：对于任何二维张量X，当满足：1. 内核的大小是奇数；2. 所有边的填充行数和列数相同；3. 输出与输入具有相同高度和宽度，则可以得出：输出Y[i, j]是通过以输入X[i, j]为中心，与卷积核进行**互相关计算**得到的。

In [4]:
def comp_conv2d(conv2d, X):
    # reshape成4阶（batch维，通道维，X.shape）
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 忽略前两维
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)          # 3*3的核
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

当卷积内核的高度和宽度不同时，我们可以填充不同的高度和宽度。

In [5]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### 2.步幅

- 步幅：卷积核窗口每次滑动的元素个数；
- 不能整除时，向下取整；即如果走不够一步，则不走这一步；
- 可以调整宽度和高度上的步幅不一致，但实际中很少使用不一致的步幅；

In [6]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape        # 4.5取下整了

torch.Size([4, 4])

In [7]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

### 3.小结

- 填充和步幅是卷积层的超参数；
- 填充在输入周围添加额外的0行和0列；
- 步幅是每次滑动核窗口时的行/列的步长。

输入维度$n_h*n_w$，卷积核维度$k_h*k_w$，行共填充$p_h$列共填充$p_w$，行步幅$s_h$列步幅$s_w$，输出维度：
$$
\lfloor \frac{n_h-k_h+p_h}{s_h}+1 \rfloor \times \lfloor \frac{n_w-k_w+p_w}{s_w}+1 \rfloor
$$